# Chapter 7. 함수 데커레이터와 클로저
## 7.1 데커레이터 기본 지식
데커레이터는 다른 함수 (데커레이터된 함수)를 인수로 받는 콜러블이다.
- 데커레이터된 함수에 어떤 처리를 수행하고, 함수를 반환하거나 다른 함수나 콜러블 객체로 대체할 수 있다.

<br>

~~~python
@decorate
def target():
    pass
~~~

<br>

위 코드는 다음과 같이 작동한다.

~~~python
decorate(target)
~~~

<br>

기본적으로 데커레이터는 함수를 인수로 전달하여 호출하는 일반적인 콜러블이지만, 런타임에 프로그램 행위를 변경하는 **메타프로그래밍**을 할 때 데커레이터가 편리하다.

In [1]:
# 예제 7-1 일반적으로는 데커레이터는 함수를 다른 함수로 대체한다.
def deco(func):
    def inner():
        print("Running inner()")
        
    return inner


@deco
def target():
    print("Running target()")
    
    

In [2]:
target()  # 데커레이트된 target()을 호출하면 실제로는 inner()를 실행한다.

Running inner()


In [3]:
target  # inner()를 가리키고 있다.

<function __main__.deco.<locals>.inner()>

## 7.2 파이썬이 데커레이터를 실행하는 시점
- 데커레이터는 **데커레이트된 함수가 정의된 시점**에 실행된다. 더 자세히는 파이썬이 모듈을 로딩하는 시점, 즉 **임포트 타임**에 실행된다.
- 데커레이터된 함수는 명시적으로 호출될 때만 실행된다.
- 일반적으로 데커레이터와 데커레이터되는 함수와 서로 다른 파일에서 정의된다.
- 일반적으로 데커레이터에서 반환할 함수는 데커레이터 내부에서 정의한다.

In [4]:
from registration import *

Running register(<function f1 at 0x114347940>)
Running register(<function f2 at 0x114094790>)


In [5]:
registry

[<function registration.f1()>, <function registration.f2()>]

In [6]:
@register
def f4():
    print("Running f()")

Running register(<function f4 at 0x114dd58b0>)


In [7]:
registry

[<function registration.f1()>,
 <function registration.f2()>,
 <function __main__.f4()>]

## 7.3 데커레이터로 개선한 전략 패턴
6장 공부하고 재방문할 예정

## 7.4 변수 범위 규칙

In [8]:
# 예제 7-4 지역 및 전역 변수를 읽는 함수
def f1(a):
    print(a)
    print(b)


In [9]:
f1(3)

3


NameError: name 'b' is not defined

In [10]:
b = 6
f1(3)

3
6


In [11]:
# 예제 7-5 함수 본체 안에서 값을 할당하기 때문에 지역 변수가 되는 b
b = 6
def f2(a):
    print(a)
    print(b)
    
    b = 9

`print(a)`는 실행되었고, 다음 `print(b)`는 전역 변수 `b`가 정의되어 있을 뿐만 아니라 지역 변수 정의가 이후에도 있음에도 지역 변수로 인식한다!

- 컴파일 단계에서 지역변수로 판단
- 파이썬은 변수가 선언되어 있기를 요구하지는 않지만, 함수 보네 안에서 할당된 변수는 지역 변수로 판단한다.

In [12]:
f2(3)

3


UnboundLocalError: local variable 'b' referenced before assignment

함수 안에서 전역 변수의 할당문을 다루고 싶다면 `global` 키워드를 이용해서 선언해야 한다.

In [13]:
b = 6
def f3(a):
    global b
    print(a)
    print(b)
    
    b = 9

In [14]:
f3(3)

3
6


In [15]:
b

9

In [16]:
b = 30
b

30

## 7.5 클로저

In [17]:
# 예제 7-8 이동 평균을 계산하는 클래스
class Averager():
    def __init__(self):
        self.series = []
        
    def __call__(self, new_value):
        self.series.append(new_value)
        total = sum(self.series)
        return total / len(self.series)
    
avg = Averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


In [18]:
# 예제 7-10 이동 평균을 계산하는 고위 함수
def make_averager():
    series = []
    
    def averager(new_value):
        series.append(new_value)
        total = sum(series)
        return total / len(series)
    
    return averager  # 함수를 반환해서 고위 함수

<span style="color: red">어떻게 series를 갖고 있는 것일까? averager 안에 정의되어 있어서 기억하는 것일까? 어떻게 접근하는거지?</span>

In [19]:
# 예제 7-10 [예제 7-9] 테스트
avg = make_averager()  

print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


`make_averager()` 안에 있는 `series`는 자유 변수 (free variable)이다.
- 자유 변수는 지역 변수에 바인딩되어 있지 않은 변수이다.

In [20]:
# 예제 7-11 [예제 7-9]의 make_averager()로 생성한 함수 조사하기
avg.__code__.co_varnames

('new_value', 'total')

In [21]:
avg.__code__.co_freevars

('series',)

In [22]:
# 예제 7-11 [예제 7-10]에서 계속
avg.__closure__

(<cell at 0x1149be580: list object at 0x11528f3c0>,)

In [23]:
avg.__closure__[0].cell_contents

[10, 11, 12]

- 클로저는 함수를 정의할 때, 존재하던 자유 변수에 대한 바인딩을 유지하는 함수
- 함수를 정의하는 범위가 사라진 후에 함수를 호출해도 자유 변수에 접근할 수 있다.
- 함수가 전역 변수가 아닌 외부 변수를 다루는 경우는, 그 함수가 다른 함수 안에 정의된 경우 뿐이다.

## 7.6 nonlocal 선언

In [24]:
# 예 7-13 전체 이력을 유지하지 않고 이동 평균을 계산하는 잘못된 고위함수
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        # count/total 변수 할당문으로서 count/total을 지역 변수로 만들어 버린다.
        count += 1  
        total += new_value
        return total / count
    
    return averager

In [25]:
avg = make_averager()
avg(10)

UnboundLocalError: local variable 'count' referenced before assignment

- `nonlocal`로 선언하면 함수 안에서 변수에ㅔ 새로운 값을 할당하더라도 그 변수는 자유 변수임을 나타낸다.

In [26]:
# 예 7-14 전체 이력을 유지하지 않고 이동 평균을 계산하기
def make_averager():
    count = 0
    total = 0
    
    def averager(new_value):
        nonlocal count, total
        count += 1  
        total += new_value
        return total / count
    
    return averager

In [27]:
avg = make_averager()
print(avg(10))
print(avg(11))
print(avg(12))

10.0
10.5
11.0


## 7.7 간단한 데커레이터 구현하기
- 일반적으로 데커레이터 함수는 데커레이터된 함수를 동일한 인수를 받는 함수로 교체하고, 동일한 값을 반환하면서, 추가적인 처리를 수행한다.


In [28]:
# 예제 7-15 함수의 실행 시간을 출력하는 간단한 데커레이터
import time


def clock(func):
    def clocked(*args):
        t0 = time.perf_counter()
        result = func(*args)
        elapsed = time.perf_counter() - t0
        
        name = func.__name__
        arg_str = ', '.join(repr(arg) for arg in args)
        print(f'[{elapsed:.8f} {name}({arg_str}) -> {result}]')
        return result
    
    return clocked

In [29]:
# 예제 7-16 clock 데커레이터 사용하기
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*' * 40, 'Calling snooze(.123')
snooze(.123)

print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123
[0.12804363 snooze(0.123) -> None]
**************************************** Calling factorial(6)
[0.00000017 factorial(1) -> 1]
[0.00000967 factorial(2) -> 2]
[0.00001496 factorial(3) -> 6]
[0.00002200 factorial(4) -> 24]
[0.00002796 factorial(5) -> 120]
[0.00003350 factorial(6) -> 720]
6! = 720


데커레이터된 함수의 `__name__`과 `__doc__` 속성을 가린다.

In [30]:
factorial

<function __main__.clock.<locals>.clocked(*args)>

In [31]:
# 예제 7-17 개선된 clock 데커레이터
import time
import functools

def clock(func):
    @functools.wraps(func)
    def clocked(*args, **kwargs):
        t0 = time.time()
        result = func(*args, **kwargs)
        elapsed = time.time() - t0
        name = func.__name__
        arg_lst = []
        if args:
            arg_lst.append(', '.join(repr(arg) for arg in args))
        if kwargs:
            pairs = [f'{k}={w}' for k, w in sorted(kwargs.items())]
            arg_lst.append(pairs)
        arg_str = ', '.join(arg_lst)
        print(f'[{elapsed:.8f}] {name}({arg_str}) -> {result}')
        return result
    return clocked
        

In [32]:
# 예제 7-16 clock 데커레이터 사용하기
@clock
def snooze(seconds):
    time.sleep(seconds)
    
@clock
def factorial(n):
    return 1 if n < 2 else n * factorial(n-1)

print('*' * 40, 'Calling snooze(.123')
snooze(.123)

print('*' * 40, 'Calling factorial(6)')
print('6! =', factorial(6))

**************************************** Calling snooze(.123
[0.12405705] snooze(0.123) -> None
**************************************** Calling factorial(6)
[0.00000000] factorial(1) -> 1
[0.00001884] factorial(2) -> 2
[0.00002670] factorial(3) -> 6
[0.00003362] factorial(4) -> 24
[0.00004101] factorial(5) -> 120
[0.00004697] factorial(6) -> 720
6! = 720


In [33]:
factorial

<function __main__.factorial(n)>

## 7.8 표준 라이브러리에서 제공하는 데커레이터
- 메서드 데커레이트를 위한 내장 함수: `property()`, `classmethod()`, `staticmethod()`
- `functools` 표준 라이브러리에서 자주 사용되는 데커레이터: `wraps()`, `lru_cache()`, `singledispatch()`

### 7.8.1 functools.lru_cache()를 이용한 메모이제이션
- 메모이제이션 (memoization)
  - <span style="color: green">컴퓨터 프로그램이 동일한 계산을 반복해야 할 때, 이전에 계산한 값을 메모리에 저장함으로써 동일한 계산의 반복 수행을 제거하여 프로그램 실행 속도를 빠르게 하는 기술.</span>
- lru: Least Recently Used


In [34]:
# 예제 7-18 피보나치 수열에서 n번째 숫자를 아주 값비싸게 계산하는 방식
@clock
def fibonacci(n):
    if n < 2:
        return n
    
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(6))

[0.00000000] fibonacci(0) -> 0
[0.00000095] fibonacci(1) -> 1
[0.00004220] fibonacci(2) -> 1
[0.00000119] fibonacci(1) -> 1
[0.00000215] fibonacci(0) -> 0
[0.00000095] fibonacci(1) -> 1
[0.00009179] fibonacci(2) -> 1
[0.00012016] fibonacci(3) -> 2
[0.00021791] fibonacci(4) -> 3
[0.00000119] fibonacci(1) -> 1
[0.00000000] fibonacci(0) -> 0
[0.00000000] fibonacci(1) -> 1
[0.00002813] fibonacci(2) -> 1
[0.00005507] fibonacci(3) -> 2
[0.00000000] fibonacci(0) -> 0
[0.00000000] fibonacci(1) -> 1
[0.00001407] fibonacci(2) -> 1
[0.00000095] fibonacci(1) -> 1
[0.00000119] fibonacci(0) -> 0
[0.00000000] fibonacci(1) -> 1
[0.00004411] fibonacci(2) -> 1
[0.00008011] fibonacci(3) -> 2
[0.00012803] fibonacci(4) -> 3
[0.00022292] fibonacci(5) -> 5
[0.00062966] fibonacci(6) -> 8
8


`maxsize`: 얼마나 많은 호출을 저장할지 결정

`typed`: True일 경우 인수즤 자료형이 다르면 결과를 따로 저장


In [35]:
# 예제 7-19 캐시를 이용한 더 빠른 구현
import functools

@functools.lru_cache(maxsize=128, typed=False)
@clock
def fibonacci(n):
    if n < 2:
        return n
    return fibonacci(n - 2) + fibonacci(n - 1)

print(fibonacci(6))

[0.00000000] fibonacci(0) -> 0
[0.00000000] fibonacci(1) -> 1
[0.00004005] fibonacci(2) -> 1
[0.00000000] fibonacci(3) -> 2
[0.00005722] fibonacci(4) -> 3
[0.00000095] fibonacci(5) -> 5
[0.00008917] fibonacci(6) -> 8
8


### 7.8.2 단일 디스패치를 이용한 범용 함수

In [36]:
# 예제 7-20 다른 객체형에 맞춰진 HTML을 생성하는 htmlize()
import html

def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

print(htmlize({1, 2, 3}))
print(htmlize(abs))
print(htmlize('Heimlich & Co.\n a game'))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<pre>&#x27;Heimlich &amp; Co.\n a game&#x27;</pre>
<pre>42</pre>
<pre>[&#x27;alpha&#x27;, 66, {1, 2, 3}]</pre>


- 일반 함수를 `@singledispatch`로 데커레이트하면, 이 함수는 범용 함수가 된다.
- 일련의 함수가 첫 번째 인수의 자료형에 따라 서로 다른 방식으로 연산을 수행하게 된다.


In [37]:
# 예제 7-21 여러 함수를 범용 함수로 묶는 커스텀 htmlize.register()를 생성하는 singledispatch
from functools import singledispatch
from collections import abc
import numbers
import html

@singledispatch
def htmlize(obj):
    content = html.escape(repr(obj))
    return f'<pre>{content}</pre>'

@htmlize.register(str)
def _(text):
    content = html.escape('text').replace('\n', '<br>\n')
    return f'<p>{content}</p>'

@htmlize.register(numbers.Integral)
def _(n):
    return f'<p>{n} (0x{n:x})</p>'

@htmlize.register(tuple)
@htmlize.register(abc.MutableSequence)
def _(seq):
    inner = '<li>\n</li>'.join(htmlize(item) for item in seq)
    return '<ul>\n<li>' + inner + '</li>\n</ul>'

In [38]:
print(htmlize({1, 2, 3}))
print(htmlize(abs))
print(htmlize('Heimlich & Co.\n a game'))
print(htmlize(42))
print(htmlize(['alpha', 66, {3, 2, 1}]))

<pre>{1, 2, 3}</pre>
<pre>&lt;built-in function abs&gt;</pre>
<p>text</p>
<p>42 (0x2a)</p>
<ul>
<li><p>text</p><li>
</li><p>66 (0x42)</p><li>
</li><pre>{1, 2, 3}</pre></li>
</ul>


## 7.9 누적된 데커레이터

~~~python
@d1
@d2
def f():
    print('f')
~~~

<br>

~~~python
def f():
    print('f')

f = d1(d2(f))
~~~

## 7.10 매개변수화된 데커레이터
함수 외에 추가적인 다른 인수를 받는 데커레이터를 만드는 방법
- 데커레이터를 반환하는 데커레이터 팩토리를 만들고 나서, 데커레이트될 함수에 데커레이터 팩토리를 적용하면 된다.

In [46]:
# 예제 7-22 [예제 7-2]의 축약 버전
registry = []
def register(func):
    print(f"running register({func})")
    registry.append(func)
    return func


@register
def f1():
    print("running f1()")
    
print("resigtry -> ", registry)
f1()    

running register(<function f1 at 0x1152b69d0>)
resigtry ->  [<function f1 at 0x1152b69d0>]
running f1()


In [49]:
# 예제 7-23 매개변수를 받기 위해 함수로 호출되어야 하는 새로운 register() 데커레이터
registry = set()

def register(active=True):
    def decorate(func):
        print(f"Running register(active={active}) -> decorate({func})")
        if active:
            registry.add(func)
        else:
            registry.discard(func)
        
        return func
    return decorate


@register(active=False)
def f1():
    print("Running f1()")
    
@register(active=True)
def f2():
    print("Running f2()")
    
def f3():
    print("Running f3()")

Running register(active=False) -> decorate(<function f1 at 0x115122e50>)
Running register(active=True) -> decorate(<function f2 at 0x1152b6280>)


`register()`가 `decorate()`를 반환하고, 데커레이트될 함수에 `decorate()`가 적용된다.

In [50]:
# 예제 7-24 [예제 7-23]에 나열된 registration_param 모듈 사용하기
print(registry)

register()(f3)
print(registry)

register(active=False)(f2)
print(registry)

{<function f2 at 0x1152b6280>}
Running register(active=True) -> decorate(<function f3 at 0x1152b69d0>)
{<function f2 at 0x1152b6280>, <function f3 at 0x1152b69d0>}
Running register(active=False) -> decorate(<function f2 at 0x1152b6280>)
{<function f3 at 0x1152b69d0>}


<span style="color: green">첫 번째 인자로 자동으로 데커레이터될 함수가 들어간다면 다음과 같이 되지 않을까?</span>

In [53]:
registry = []

def register(func, active=True):
    print(f"running register({func}, active={active})")
    registry.append(func)
    return func


@register
def f1():
    print("running f1()")
    
@register(active=False)
def f2():
    print("running f1()")
    
print("resigtry -> ", registry)
f1()    

running register(<function f1 at 0x1152b6700>, active=True)


TypeError: register() missing 1 required positional argument: 'func'

<span style="color: green">
내가 생각하는 결론은 다음과 같다. @ 뒤에는 콜러블 객체가 와야 한다.<br>
@함수() 이런 식으로 된 순간 함수는 수행된 것이고, return된 객체로 바뀌게 된다.<br>
return된 객체가 콜러블 객체이면, 콜러블객체(데코레이트된 함수)가 수행되는 것 같다</span>

### 7.10.2 매개변수화된 clock 데커레이터

In [59]:
# 예제 7-25 매개변수화된 clock() 데커레이터
import time

DEFAULT_FMT = '[{elapsed:0.8f}s] {name}({args}) -> {result}'

def clock(fmt=DEFAULT_FMT):
    def decorate(func):
        def clocked(*_args):
            t0 = time.time()
            _result = func(*_args)
            elapsed = time.time() - t0
            name = func.__name__
            args = ', '.join(repr(arg) for arg in _args)
            result = repr(_result)

            print(fmt.format(**locals()))
            return _result   
        return clocked
    return decorate


@clock()
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

[0.12417507s] snooze(0.123) -> None
[0.12653112s] snooze(0.123) -> None
[0.12490606s] snooze(0.123) -> None


In [60]:
# 예제 7-26 clock 데모1
import time

@clock('{name}: {elapsed}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)

snooze: 0.12565302848815918s
snooze: 0.12601423263549805s
snooze: 0.1280219554901123s


In [62]:
# 예제 7-27 clock 데모2
import time

@clock('{name}({args}) dt={elapsed:0.3f}s')
def snooze(seconds):
    time.sleep(seconds)
    
for i in range(3):
    snooze(.123)


snooze(0.123) dt=0.128s
snooze(0.123) dt=0.128s
snooze(0.123) dt=0.128s
